In [ ]:
def f_her_infogram(x_cal, y_cal, z_cal, her):
    # function to extract the Spatial structure of the data
    # -------------- Input --------------
    # - x_cal     [N,1]   x coordidates of the calibration set
    # - y_cal     [N,1]   y coordidates of the calibration set
    # - z_cal     [N,1]   z values of the calibration set (variable under study)
    # - her       struct  structure cointaining model definitions
    # -------------- Output --------------
    # - her       struct  structure cointaining model definitions
    # -------------- Version --------------
    # - 2020/04/10 Stephanie Thiesen: intial version
    # -------------- Script --------------
    # Geo1: Spatial Characterization
    # step 1: get diff_z as a function of the distance 
    # calculate the euclidean distance and the difference between all point pairs
    zm_res_ = np.min(np.diff(np.unique(z_cal))) # smallest Z resolution
    her.n_bins_z_per_bin_shift = min(100, round(0+her.binwidth_z / zm_res_)) # limit to 100 or binwidth_shift = 1 (in the case where all zm are integers)
    her.binwidth_shift = her.binwidth_z / her.n_bins_z_per_bin_shift
    her.mat_euc_distance_xy = np.empty((len(x_cal), len(x_cal))) # matrix for the distance between 2 pair points
    her.mat_diff_z = np.empty((len(z_cal), len(z_cal))) # matrix for the diff_z 
    for i in range(len(x_cal)): # repeat to each east coordinates (x)
        for j in range(len(x_cal)): # repeat to each north coordinates (y)
            her.mat_euc_distance_xy[i,j] = f_euclidean_dist(x_cal[i], y_cal[i], x_cal[j], y_cal[j]) # calculate the euclidean distance
            her.mat_diff_z[i,j] = f_diff(z_cal[i], z_cal[j]) # calculate the diff_z
    
    # step 2: define the distance classes (lag)
    her.n_lag = np.ceil(np.max(her.mat_euc_distance_xy) / her.lag_dist) # total number of classes
    her.edges_distance_classes = np.arange(0, her.lag_dist*(her.n_lag+1), her.lag_dist) # edges of the distance classes or DEFINE  it
    
    her.bin_centers_distance_classes = np.empty(her.n_lag) # bin centers of the distance classes
    # check empty classes
    count = 0
    her.emptyclass = []
    for i in range(her.n_lag): # for each class
        idx = np.logical_and(her.mat_euc_distance_xy > her.edges_distance_classes[i], her.mat_euc_distance_xy <= her.edges_distance_classes[i+1]) # find the diff_z within the current lag class
        her.bin_centers_distance_classes[i] = (her.edges_distance_classes[i] + her.edges_distance_classes[i+1])/2
        if np.sum(idx) == 0:
            her.emptyclass.append(i)
    
    if len(her.emptyclass) > 0: # if we have empty bins, inform
        str = 'Atention! There are empty distance classes (' + ', '.join(map(str, her.emptyclass)) + '). Consider increase the lag or bin width to fill the classes into the range.'
        print(str)
    
    # step 3: compute bin edges
    max_diff_z = np.max(her.mat_diff_z)
    n_bins = 2*np.floor(max_diff_z/her.binwidth + 1) # symmetric
    mini = -n_bins/2*her.binwidth
    maxi =  n_bins/2*her.binwidth
    her.edges_diff_z = np.linspace(mini, maxi, n_bins+1) # edges of the diff_z pmf
    her.n_bins_shift = 2*np.floor(max_diff_z/her.binwidth_shift + 1) # symmetric
    mini = -her.n_bins_shift/2*her.binwidth_shift
    maxi =  her.n_bins_shift/2*her.binwidth_shift
    her.edges_diff_z_shift = np.linspace(mini, maxi, her.n_bins_shift+1) # edges of the smaller diff_z pmf
    
    # step 4: Compute delta_z PMF for the full dataset
    idx = her.mat_euc_distance_xy > 0 # ignore the index of the observation in relation to itself
    her.obs_diff_z = her.mat_diff_z[idx] # take the diff_z observations
    pmf_diff_z_all_obs_withemptybins, _ = np.histogram(her.obs_diff_z, bins=her.edges_diff_z, density=True)
    countpmf_diff_z_all_obs, _ = np.histogram(her.obs_diff_z, bins=her.edges_diff_z)
    countpmf_diff_z_all_obs_plus1 = countpmf_diff_z_all_obs + 1 # avoiding empty bins (PLUS 1 in each bin)
    pmf_diff_z_all_obs = countpmf_diff_z_all_obs_plus1 / np.sum(countpmf_diff_z_all_obs_plus1)
    prob_min = 1/(len(her.obs_diff_z))
    
    countpmf_diff_z_all_obs_shift, _ = np.histogram(her.obs_diff_z, bins=her.edges_diff_z_shift)
    countpmf_diff_z_all_obs_shift_plus1 = countpmf_diff_z_all_obs_shift + 1/her.n_bins_z_per_bin_shift # avoiding empty bins (PLUS 1/n in each bin)
    countpmf_diff_z_all_obs_shift_plus1 = np.concatenate(([1/her.n_bins_z_per_bin_shift], countpmf_diff_z_all_obs_shift_plus1)) # add empty bin in beginning
    pmf_diff_z_all_obs_shift = countpmf_diff_z_all_obs_shift_plus1 / np.sum(countpmf_diff_z_all_obs_shift_plus1)
    prob_min_shift = prob_min/her.n_bins_z_per_bin_shift
    
    # check probabilities sum to 1
    if np.abs(np.sum(pmf_diff_z_all_obs) - 1) > .00001:
        raise ValueError('Probablities dont sum to 1.')
    
    if np.abs(np.sum(pmf_diff_z_all_obs_shift) - 1) > .00001:
        raise ValueError('Probablities dont sum to 1.')
    
    # step 5: Calculate the delta_z PMF by distance class
    # address delta_z observations to its distance classes
    obs_diff_z_by_class = []
    pmf_diff_z_by_class_obs_withemptybins = np.empty((len(her.edges_distance_classes)-1, n_bins))
    pmf_diff_z_by_class_obs = np.empty((len(her.edges_distance_classes)-1, n_bins)) # diff_z PMF by lag class (each row is one class, the columns are the probability of the diff_z bins)
    pmf_diff_z_by_class_obs_shift_withemptybins = np.empty((len(her.edges_distance_classes)-1, her.n_bins_shift))
    pmf_diff_z_by_class_obs_shift = np.empty((len(her.edges_distance_classes)-1, her.n_bins_shift+1)) # diff_z PMF by lag class (each row is one class, the columns are the probability of the diff_z bins)
    for i in range(len(her.edges_distance_classes)-1): # for each class
        idx = np.logical_and(her.mat_euc_distance_xy > her.edges_distance_classes[i], her.mat_euc_distance_xy <= her.edges_distance_classes[i+1]) # find the diff_z within the current lag class
        obs_diff_z_by_class.append(her.mat_diff_z[idx]) # save the diff_z 
        pmf_diff_z_by_class_obs_withemptybins[i], _ = np.histogram(obs_diff_z_by_class[i], bins=her.edges_diff_z, density=True)
        pmf_diff_z_by_class_obs[i] = pmf_diff_z_by_class_obs_withemptybins[i] + prob_min
        pmf_diff_z_by_class_obs[i] = pmf_diff_z_by_class_obs[i] / np.sum(pmf_diff_z_by_class_obs[i]) # normalization
        pmf_diff_z_by_class_obs_shift_withemptybins[i], _ = np.histogram(obs_diff_z_by_class[i], bins=her.edges_diff_z_shift, density=True)
        pmf_diff_z_by_class_obs_shift[i,1:] = pmf_diff_z_by_class_obs_shift_withemptybins[i] + prob_min_shift
        pmf_diff_z_by_class_obs_shift[i,1:] = pmf_diff_z_by_class_obs_shift[i,1:] / np.sum(pmf_diff_z_by_class_obs_shift[i,1:])
    
    pmf_diff_z_by_class_obs_shift[:,0] = prob_min_shift # add empty bin in beginning
    
    # check probabilities sum to 1
    for i in range(len(her.edges_distance_classes)-1): # for each class
        if np.abs(np.sum(pmf_diff_z_by_class_obs[i]) - 1) > .00001:
            raise ValueError('Probablities dont sum to 1.')
        
        if np.abs(np.sum(pmf_diff_z_by_class_obs_shift[i,1:]) - 1) > .00001:
            raise ValueError('Probablities dont sum to 1.')
    
    # step 6: Entropy
    # calculate the entropy of the delta_z PMFs for the full dataset and for the distance classes 
    her.H_diff_z = f_entropy(pmf_diff_z_all_obs) # entropy of the diff_z PMF of the full dataset
    her.H_diff_z_by_class = np.empty((len(her.edges_distance_classes)-1,1)) # vector for the entropy of the diff_z PMF by lag classes 
    for i in range(len(her.edges_distance_classes)-1): # for each lag class
        her.H_diff_z_by_class[i] = f_entropy(pmf_diff_z_by_class_obs[i]) # calculate the entropy
    
    # step 7: Define the info RANGE, and associate delta_z PMF of the full dataset to the distance classes beyond the range
    # calculate the PMF of the full dataset where the class entropy > full dataset entropy 
    # info RANGE: limit where the entropy of the lag class is greater than the entropy of the full dataset
    if her.H_diff_z > her.H_diff_z_by_class[~np.isnan(her.H_diff_z_by_class)].any():
        her.n_classes_limit = 20 # DEFINE it looking at the infogram
        her.H_lim = her.H_diff_z_by_class[her.n_classes_limit]
        her.n_classes_range = np.where(her.H_diff_z_by_class > her.H_lim)[0][0]
    else:
        her.H_lim = her.H_diff_z
        her.n_classes_range = np.where(her.H_diff_z_by_class > her.H_lim)[0][0] # number of distance classes inside the range + full dataset
    
    her.edges_distance_classes_range = np.array([0]) # edges of distance classes considering the range (starting in zero meters)
    her.bin_centers_distance_classes_range = np.empty(0) # bin centers of the distance classes
    her.pmf_diff_z_by_class_obs_range = np.empty((0, n_bins)) # diff_z PMF by lag class considering the range (each row is one class, the columns are the probability of the diff_z bins)
    her.H_diff_z_by_class_range = np.empty(0) # entropy of the delta_z PMF by lag classes inside the range + full dataset
    obs_diff_z_by_class_range = [] # observations of diff_z by lag class considering the range (each cell is one lag class) 
    for i in range(pmf_diff_z_by_class_obs.shape[0]): # for each class
        if her.H_diff_z_by_class[i] > her.H_lim: # stop if the entropy of the lag class is greater than the entropy of the full dataset (or a defined one)
            break
        else:
            her.pmf_diff_z_by_class_obs_range = np.vstack((her.pmf_diff_z_by_class_obs_range, pmf_diff_z_by_class_obs[i])) # save the diff_z PMF
            her.pmf_diff_z_by_class_obs_range_shift = np.vstack((her.pmf_diff_z_by_class_obs_range_shift, pmf_diff_z_by_class_obs_shift[i])) # save the diff_z PMF
            her.edges_distance_classes_range = np.append(her.edges_distance_classes_range, her.edges_distance_classes[i+1]) # save the edge of the distance class   
            her.H_diff_z_by_class_range = np.append(her.H_diff_z_by_class_range, her.H_diff_z_by_class[i]) # save the entropy of the diff_z PMF
            obs_diff_z_by_class_range.append(obs_diff_z_by_class[i]) # save the diff_z observation
    
    # create a last distance class and associate to it 
    her.pmf_diff_z_by_class_obs_range = np.vstack((her.pmf_diff_z_by_class_obs_range, pmf_diff_z_all_obs)) # diff_z PMF of the full dataset
    her.pmf_diff_z_by_class_obs_range_shift = np.vstack((her.pmf_diff_z_by_class_obs_range_shift, pmf_diff_z_all_obs_shift)) # diff_z PMF of the full dataset
    her.edges_distance_classes_range = np.append(her.edges_distance_classes_range, her.edges_distance_classes[-1]) # the right edge of the distance class 
    her.H_diff_z_by_class_range = np.append(her.H_diff_z_by_class_range, her.H_diff_z) # the entropy of the diff_z PMF of the full dataset
    obs_diff_z_by_class_range.append(her.obs_diff_z) # the diff_z of the full dataset
    
    # calculate bin centers of the distance classes
    her.bin_centers_distance_classes_range = 1/2 * (her.edges_distance_classes_range[:-1] + her.edges_distance_classes_range[1:])
    
    # distribution of the # of pairs by class (used for the histogram construction)
    her.n_pairs_by_class = np.empty(her.n_lag)
    for i in range(her.n_lag):
        her.n_pairs_by_class[i] = len(obs_diff_z_by_class[i])
    
    if np.isnan(np.sum(her.pmf_diff_z_by_class_obs_range)):
        str = 'Atention! There are no histograms for \Deltaz in the range (classes without histogram: ' + ', '.join(map(str, np.where(np.sum(her.pmf_diff_z_by_class_obs_range, axis=1) != 1)[0])) + '). Interpolation not possivel. Consider increase \Deltaz binwidth.'
        print(str)
    
    # For Geo2: define edges of Z 
    max_z = np.max(z_cal)
    min_z = np.min(z_cal)
    max_diff_z = her.edges_diff_z_shift[-1]
    min_diff_z = her.edges_diff_z_shift[0]
    # compute bin edges
    n_bins_left = -np.floor((min_diff_z + min_z)/ her.binwidth_z)
    n_bins_right = np.floor((max_diff_z + max_z)/ her.binwidth_z + 1) # edge <= x < edge
    her.n_bins_z = int(n_bins_left+n_bins_right)
    mini = -her.binwidth_z*n_bins_left
    maxi =  her.binwidth_z*n_bins_right
    her.edges_z = np.linspace(mini, maxi, her.n_bins_z+1) # edges of the z+diff_z pmf
    pmf_z_all_obs, _ = np.histogram(z_cal, bins=her.edges_z, density=True) # calculate the z PMF of the full dataset 
    her.bin_centers_edges_z = her.edges_z[:-1] + her.binwidth_z/2
    her.bin_centers_edges_diff_z = her.edges_diff_z[:-1] + her.binwidth/2
    
    return her
